In [ ]:
!pip install pyspark==3.0.3


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MySparkSession") \
    .getOrCreate()


In [ ]:
pip install findspark

In [ ]:
!pip install pyspark


In [ ]:
import os
os.environ["SPARK_HOME"] = "/path/to/spark"


In [ ]:
import findspark
findspark.init()


In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("MySparkSession").getOrCreate()

# Define the file path
file_path = '/home_sales_revised.csv'  # Replace with the correct path if needed

# Read the data into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Create a temporary view of the DataFrame
df.createOrReplaceTempView('home_sales')

#Show Dataframe
df.show()


+--------------------+--------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|    date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+--------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|  4/8/22|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...| 6/13/21|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...| 4/12/19|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|10/16/19|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|  1/8/22|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...| 1/30/19|      2017|218712|       2|        3|       1965|   14375|     2|

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

q = """
    select year(date) as year_sold, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 4
    group by year_sold
    order by year_sold
"""
spark.sql(q).show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     null|299661.01|
+---------+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
q1 = """
    select date_built as year_built, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 3 and bathrooms = 3
    group by year_built
    order by year_built
"""
spark.sql(q1).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
q2 = """
    select date_built as year_built, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
    group by year_built
    order by year_built
"""
spark.sql(q2).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

q3 = """
    select view, round(avg(price), 2) as avg_price
    from home_sales
    group by view
    having avg(price) >= 350000
    order by view
"""
spark.sql(q3).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 2.4195051193237305 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(q3).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 3.0278804302215576 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('built_partitioned')


In [ ]:
# 11. Read the formatted parquet data.
p_df_p = spark.read.parquet('built_partitioned')


In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_built_p')


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql(q3).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 1.8155295848846436 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')


DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False